In [68]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import numpy as np


# **Data Cleaning**

In [70]:
df=pd.read_csv('Bengaluru_House_Data.csv')
df.head(10)

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00
5,Super built-up Area,Ready To Move,Whitefield,2 BHK,DuenaTa,1170,2.0,1.0,38.00
6,Super built-up Area,18-May,Old Airport Road,4 BHK,Jaades,2732,4.0,NaN,204.00
7,Super built-up Area,Ready To Move,Rajaji Nagar,4 BHK,Brway G,3300,4.0,NaN,600.00
8,Super built-up Area,Ready To Move,Marathahalli,3 BHK,NaN,1310,3.0,1.0,63.25
9,Plot Area,Ready To Move,Gandhi Bazar,6 Bedroom,NaN,1020,6.0,NaN,370.00


In [72]:
df.shape


(13320, 9)

In [74]:
df.groupby('area_type')['area_type'].agg('count')

area_type
Built-up  Area          2418
Carpet  Area              87
Plot  Area              2025
Super built-up  Area    8790
Name: area_type, dtype: int64

drop columns that are not required

In [76]:
df2 = df.drop(['area_type', 'society', 'balcony', 'availability'], axis='columns')
df2.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


removing rows with null values**

In [78]:
df2.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [80]:
df3=df2.dropna()
df3.head()
df3.isnull().sum()
df3.shape

(13246, 5)

In [82]:
df3['size']

0            2 BHK
1        4 Bedroom
2            3 BHK
3            3 BHK
4            2 BHK
           ...    
13315    5 Bedroom
13316        4 BHK
13317        2 BHK
13318        4 BHK
13319        1 BHK
Name: size, Length: 13246, dtype: object

there is inconsistency present in size like bedroom somewhere its bhk

In [84]:
# df3['bhk']=df3['size'].apply(lambda x:int(x.split(' ')[0]))
a = df3['size'].apply(lambda x: int(x.split(' ')[0]))
print(a)
df3=df3.drop(['size'], axis='columns')

df3['bhk']=a
df3.head()

0        2
1        4
2        3
3        3
4        2
        ..
13315    5
13316    4
13317    2
13318    4
13319    1
Name: size, Length: 13246, dtype: int64


,location,total_sqft,bath,price,bhk
0,Electronic City Phase II,1056,2.0,39.07,2
1,Chikka Tirupathi,2600,5.0,120.00,4
2,Uttarahalli,1440,2.0,62.00,3
3,Lingadheeranahalli,1521,3.0,95.00,3
4,Kothanur,1200,2.0,51.00,2


In [85]:

df3[df3['bhk']>20]

,location,total_sqft,bath,price,bhk
1718,2Electronic City Phase II,8000,27.0,230.0,27
4684,Munnekollal,2400,40.0,660.0,43


In [86]:
df3.loc[30]

location        Yelahanka
total_sqft    2100 - 2850
bath                  4.0
price               186.0
bhk                     4
Name: 30, dtype: object

**some values of square feet are in range so they need to be resolved**

In [87]:
def ifloat(x):
    try:
        float(x)
    except:
        return False
    return True

In [88]:
df3[~df3['total_sqft'].apply(ifloat)].head(10)

,location,total_sqft,bath,price,bhk
30,Yelahanka,2100 - 2850,4.0,186.000,4
122,Hebbal,3067 - 8156,4.0,477.000,4
137,8th Phase JP Nagar,1042 - 1105,2.0,54.005,2
165,Sarjapur,1145 - 1340,2.0,43.490,2
188,KR Puram,1015 - 1540,2.0,56.800,2
410,Kengeri,34.46Sq. Meter,1.0,18.500,1
549,Hennur Road,1195 - 1440,2.0,63.770,2
648,Arekere,4125Perch,9.0,265.000,9
661,Yelahanka,1120 - 1145,2.0,48.130,2
672,Bettahalsoor,3090 - 5002,4.0,445.000,4


In [89]:
def sqft_to_num(x):
    t = x.split('-')
    if len(t) == 2:
        return (float(t[0])+float(t[1]))/2
    try:
        return float(x)
    except:
        return None

In [90]:
df4=df3.copy()
df4['total_sqft']=df4['total_sqft'].apply(sqft_to_num)
df4.loc[30]
df4.head(30)

,location,total_sqft,bath,price,bhk
0,Electronic City Phase II,1056.0,2.0,39.07,2
1,Chikka Tirupathi,2600.0,5.0,120.00,4
2,Uttarahalli,1440.0,2.0,62.00,3
3,Lingadheeranahalli,1521.0,3.0,95.00,3
4,Kothanur,1200.0,2.0,51.00,2
5,Whitefield,1170.0,2.0,38.00,2
6,Old Airport Road,2732.0,4.0,204.00,4
7,Rajaji Nagar,3300.0,4.0,600.00,4
8,Marathahalli,1310.0,3.0,63.25,3
9,Gandhi Bazar,1020.0,6.0,370.00,6


In [91]:
df4[~df4['total_sqft'].apply(ifloat)].head(10)

,location,total_sqft,bath,price,bhk


*all the ranges are removed from total_sqft column*

In [92]:
df5 = df4.copy()
df5['price_per_sqft'] = df5['price']*100000/df5['total_sqft']
df5.head()
df5.dropna()

,location,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,1200.0,2.0,51.00,2,4250.000000
...,...,...,...,...,...,...
13315,Whitefield,3453.0,4.0,231.00,5,6689.834926
13316,Richards Town,3600.0,5.0,400.00,4,11111.111111
13317,Raja Rajeshwari Nagar,1141.0,2.0,60.00,2,5258.545136
13318,Padmanabhanagar,4689.0,4.0,488.00,4,10407.336319


# PHASE 1 OF CLEANING ENDS HERE

# **ML part starts here**

In [93]:
dummies = pd.get_dummies(df5.location)
dummies.head()

,Anekal,Banaswadi,Basavangudi,Bhoganhalli,Devarabeesana Halli,Devarachikkanahalli,Electronic City,Mysore Highway,Rachenahalli,Sector 1 HSR Layout,...,rr nagar,sankeswari,sapthagiri Layout,sarjapura main road,singapura paradise,t.c palya,tc.palya,vinayakanagar,"white field,kadugodi",whitefiled
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
df6 = pd.concat([df5, dummies],axis='columns')
df6.head()
df6.isnull().sum()

location                 0
total_sqft              46
bath                     0
price                    0
bhk                      0
                        ..
t.c palya                0
tc.palya                 0
vinayakanagar            0
white field,kadugodi     0
whitefiled               0
Length: 1310, dtype: int64

In [95]:
df6=df6.dropna()

In [96]:
df6.isnull().sum()

location                0
total_sqft              0
bath                    0
price                   0
bhk                     0
                       ..
t.c palya               0
tc.palya                0
vinayakanagar           0
white field,kadugodi    0
whitefiled              0
Length: 1310, dtype: int64

In [97]:
df6.loc[4]['Kothanur']

1

In [98]:
df7 = df6.drop('location', axis='columns')
df7.head()


,total_sqft,bath,price,bhk,price_per_sqft,Anekal,Banaswadi,Basavangudi,Bhoganhalli,Devarabeesana Halli,...,rr nagar,sankeswari,sapthagiri Layout,sarjapura main road,singapura paradise,t.c palya,tc.palya,vinayakanagar,"white field,kadugodi",whitefiled
0,1056.0,2.0,39.07,2,3699.810606,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2600.0,5.0,120.00,4,4615.384615,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1440.0,2.0,62.00,3,4305.555556,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1521.0,3.0,95.00,3,6245.890861,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1200.0,2.0,51.00,2,4250.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
df7.shape

(13200, 1309)

In [100]:

X = df7.drop('price', axis='columns')
X.head()


,total_sqft,bath,bhk,price_per_sqft,Anekal,Banaswadi,Basavangudi,Bhoganhalli,Devarabeesana Halli,Devarachikkanahalli,...,rr nagar,sankeswari,sapthagiri Layout,sarjapura main road,singapura paradise,t.c palya,tc.palya,vinayakanagar,"white field,kadugodi",whitefiled
0,1056.0,2.0,2,3699.810606,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2600.0,5.0,4,4615.384615,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1440.0,2.0,3,4305.555556,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1521.0,3.0,3,6245.890861,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1200.0,2.0,2,4250.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [101]:
X=X.drop('price_per_sqft', axis='columns')
X.head()

,total_sqft,bath,bhk,Anekal,Banaswadi,Basavangudi,Bhoganhalli,Devarabeesana Halli,Devarachikkanahalli,Electronic City,...,rr nagar,sankeswari,sapthagiri Layout,sarjapura main road,singapura paradise,t.c palya,tc.palya,vinayakanagar,"white field,kadugodi",whitefiled
0,1056.0,2.0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2600.0,5.0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1440.0,2.0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1521.0,3.0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1200.0,2.0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
y = df7.price
y.head()

0     39.07
1    120.00
2     62.00
3     95.00
4     51.00
Name: price, dtype: float64

# **linear regression**

In [103]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [104]:
X_train.head()

,total_sqft,bath,bhk,Anekal,Banaswadi,Basavangudi,Bhoganhalli,Devarabeesana Halli,Devarachikkanahalli,Electronic City,...,rr nagar,sankeswari,sapthagiri Layout,sarjapura main road,singapura paradise,t.c palya,tc.palya,vinayakanagar,"white field,kadugodi",whitefiled
9746,2500.0,4.0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8089,3978.0,4.0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6394,1255.0,2.0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11211,1470.0,2.0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3641,2912.0,6.0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [105]:
X_train.isnull().sum()

total_sqft              0
bath                    0
bhk                     0
 Anekal                 0
 Banaswadi              0
                       ..
t.c palya               0
tc.palya                0
vinayakanagar           0
white field,kadugodi    0
whitefiled              0
Length: 1307, dtype: int64

In [106]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)

lr.score(X_test,y_test)
# lr.coef_

0.35698937711915746

# * Attempts to improve accuracy*

In [107]:
# //cleaning
df5.location = df5.location.apply(lambda x: x.strip())
df5.head()

,location,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,1200.0,2.0,51.00,2,4250.000000


In [108]:
location_stats = df5.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_stats

location
Whitefield               535
Sarjapur  Road           392
Electronic City          304
Kanakpura Road           266
Thanisandra              236
                        ... 
1 Giri Nagar               1
Kanakapura Road,           1
Kanakapura main  Road      1
Karnataka Shabarimala      1
whitefiled                 1
Name: location, Length: 1293, dtype: int64

In [109]:

location_stats_less_than_10 = location_stats[location_stats<=10]
location_stats_less_than_10

location
Basapura                 10
1st Block Koramangala    10
Gunjur Palya             10
Kalkere                  10
Sector 1 HSR Layout      10
                         ..
1 Giri Nagar              1
Kanakapura Road,          1
Kanakapura main  Road     1
Karnataka Shabarimala     1
whitefiled                1
Name: location, Length: 1052, dtype: int64

In [110]:
df5.location = df5.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
df5=df5[df5.location!='other']
df5=df5.dropna()
# dropped rows of other category
len(df5.location.unique())

241

In [111]:
df5.head()

,location,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,1200.0,2.0,51.00,2,4250.000000


# **REGRESSION**

In [112]:
dummies = pd.get_dummies(df5.location)
df6 = pd.concat([df5, dummies],axis='columns')
df6=df6.dropna()
df7 = df6.drop('location', axis='columns')
X = df7.drop('price', axis='columns')
X=X.drop('price_per_sqft', axis='columns')
y = df7.price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [113]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)

lr.score(X_test,y_test)
# lr.coef_

0.620507547985162

In [114]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
cv = ShuffleSplit(n_splits=4, test_size=0.25, random_state=0) 
cross_val_score(LinearRegression(), X,y, cv=cv)
# cv->cross validation

array([0.63168506, 0.54406482, 0.6118159 , 0.60131749])

In [115]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

# def find_best_model_using_gridsearch(X, y):
#     algos = {
#         'linear_regression': {
#             'model': LinearRegression(),
#             'params': {
#                 'normalize': [False]
#             }
#         },
#         'lasso': {
#             'model': Lasso(),
#             'params': {
#                 'alpha': [1, 2],#alpha is lambda equivalent
#                 'selection': ['random', 'cyclic']  #random -> any random coefficint is updated each iteration cyclic is default
#             }
#         },
#         'decision_tree': {
#             'model': DecisionTreeRegressor(),
#             'params': {
#                 'criterion': ['mse','mae'],
#                 'splitter': ['best', 'random']
#             }
#         }
#     }
    
#     scores = []
#     cv = ShuffleSplit(n_splits=5, test_size=0.20, random_state=1)
#     for algo_name, config in algos.items():
#         gs = GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
#         gs.fit(X, y)
#         scores.append({
#             'model': algo_name,
#             'best_score': gs.best_score_,
#             'best_params': gs.best_params_
#         })
        
#     return pd.DataFrame(scores, columns = ['model', 'best_score', 'best_params'])

# find_best_model_using_gridsearch(X, y)

In [116]:
dr=DecisionTreeRegressor(criterion='mae', splitter='best',)
dr.fit(X_train,y_train)

dr.score(X_test,y_test)

0.7371218126659289

In [117]:
print(np.where(X.columns=='1st Phase JP Nagar')[0][0])#gives column number of the location
X.columns

4


Index(['total_sqft', 'bath', 'bhk', '1st Block Jayanagar',
       '1st Phase JP Nagar', '2nd Phase Judicial Layout',
       '2nd Stage Nagarbhavi', '5th Block Hbr Layout', '5th Phase JP Nagar',
       '6th Phase JP Nagar',
       ...
       'Vijayanagar', 'Vishveshwarya Layout', 'Vishwapriya Layout',
       'Vittasandra', 'Whitefield', 'Yelachenahalli', 'Yelahanka',
       'Yelahanka New Town', 'Yelenahalli', 'Yeshwanthpur'],
      dtype='object', length=244)

In [118]:
def predict_pricing(loc,sqft,bath,bhk):
    loc_col_no=np.where(X.columns==loc)[0][0]
    x=np.zeros(len(X.columns))
    x[0]=sqft
    x[1]=bath
    x[2]=bhk
    x[loc_col_no]=1
    return dr.predict([x])[0]
    

In [130]:
predict_pricing('1st Phase JP Nagar', 1000,4 ,3)

66.0

In [119]:
predict_pricing('1st Phase JP Nagar', 1000, 2, 2)

75.0

In [120]:
predict_pricing('1st Phase JP Nagar', 1000, 3, 3)

90.0

In [121]:
predict_pricing('Indira Nagar', 1000, 2, 2)

42.25

In [122]:
predict_pricing('Whitefield', 1000, 3, 3)

90.0

# **EXPORTING MODEL AS PICKLE FILE**

In [123]:
import pickle
with open('Bengaluru_House_Data.pickle', 'wb') as f:
    pickle.dump(dr, f)

In [124]:
import json
columns = {
    'data_columns': [col.lower() for col in X.columns]
}
with open('columns.json', 'w') as f:
    f.write(json.dumps(columns))